In [ ]:
!pip install tqdm
!pip install moviepy
from fcn8s_tensorflow import FCN8s
from data_generator.batch_generator import BatchGenerator
from helpers.visualization_utils import print_segmentation_onto_image, create_video_from_images
from cityscapesscripts.helpers.labels import TRAINIDS_TO_COLORS_DICT, TRAINIDS_TO_RGBA_DICT

from math import ceil
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# TODO: Set the paths to the images.
train_images_synth = '/content/Synth2Real/Synthetic/Images/'


# TODO: Set the paths to the ground truth images.
train_gt_synth = '/content/Synth2Real/Synthetic/Labels/'

# TODO: Set the paths to the images.
train_images_real = '/content/Synth2Real/Real/Images/'


# TODO: Set the paths to the ground truth images.
train_gt_real = '/content/Synth2Real/Real/Labels/'


test_images = '/content/Synth2Real/test/Images/'

test_gt = '/content/Synth2Real/test/Labels/'

num_classes = 34 # TODO: Set the number of segmentation classes.

train_dataset_real = BatchGenerator(image_dirs=train_images_real,
                               image_file_extension='png',
                               ground_truth_dirs=train_gt_real,
                               image_name_split_separator='leftImg8bit',
                               ground_truth_suffix='gtFine_labelIds',
                               check_existence=True,
                               num_classes=num_classes)

train_dataset = [train_dataset_real]

val_dataset = BatchGenerator(image_dirs=test_images,
                             image_file_extension='png',
                             ground_truth_dirs=test_gt,
                             image_name_split_separator='leftImg8bit',
                             ground_truth_suffix='gtFine_labelIds',
                             check_existence=True,
                             num_classes=num_classes)

num_train_images = train_dataset[0].get_num_files() 
num_val_images = val_dataset.get_num_files()

print("Size of training dataset: ", num_train_images, " images")
print("Size of validation dataset: ", num_val_images, " images")

In [ ]:
# TODO: Set the batch size. I'll use the same batch size for both generators here.
batch_size = 4

train_generator = []

train_generator.append(train_dataset[0].generate(batch_size=batch_size,
                                         convert_colors_to_ids=False,
                                         convert_ids_to_ids=False,
                                         convert_to_one_hot=True,
                                         void_class_id=None,
                                         random_crop=False,
                                         crop=False,
                                         resize=False,
                                         brightness=False,
                                         flip=0.5,
                                         translate=False,
                                         scale=False,
                                         gray=False,
                                         to_disk=False,
                                         shuffle=True))

val_generator = val_dataset.generate(batch_size=batch_size,
                                     convert_colors_to_ids=False,
                                     convert_ids_to_ids=False,
                                     convert_to_one_hot=True,
                                     void_class_id=None,
                                     random_crop=False,
                                     crop=False,
                                     resize=False,
                                     brightness=False,
                                     flip=False,
                                     translate=False,
                                     scale=False,
                                     gray=False,
                                     to_disk=False,
                                     shuffle=True)

In [ ]:
import tensorflow as tf

In [ ]:
tf.reset_default_graph()
model = FCN8s(model_load_dir=None,
              tags=None,
              vgg16_dir='/content/VGG-16_mod2FCN_ImageNet-Classification',
              num_classes=num_classes,
              variables_load_dir=None)

In [ ]:
epochs = 200 # TODO: Set the number of epochs to train for.

# TODO: Define a learning rate schedule function to be passed to the `train()` method.
def learning_rate_schedule(step):
    if step <= 10000: return 0.0001
    elif 10000 < step <= 20000: return 0.00001
    elif 20000 < step <= 40000: return 0.000003
    else: return 0.000001
    
model.train(train_generator=train_generator,
            epochs=epochs,
            steps_per_epoch=ceil(num_train_images/batch_size),
            learning_rate_schedule=learning_rate_schedule,
            keep_prob=0.5,
            l2_regularization=0.0,
            eval_dataset='val',
            eval_frequency=1,
            val_generator=val_generator,
            val_steps=ceil(num_val_images/batch_size),
            metrics={'loss', 'mean_iou', 'accuracy'},
            save_during_training=True,
            save_dir='only_real',
            save_best_only=True,
            save_tags=['default'],
            save_name='(batch-size-4)',
            save_frequency=2,
            saver='saved_model',
            monitor='loss',
            record_summaries=True,
            summaries_frequency=10,
            summaries_dir='tensorboard_log/only_real',
            summaries_name='configuration_01',
            training_loss_display_averaging=3)

In [ ]:
model.evaluate(data_generator=val_generator,
               metrics={'loss', 'mean_iou', 'accuracy'},
               num_batches=ceil(num_val_images/batch_size),
               l2_regularization=0.0,
               dataset='val')

## Make predictions and visualize them

In [ ]:
images, labels = next(val_generator)

In [ ]:
n = 3 # Select which image of the batch you would like to visualize.

# Make a prediction.
prediction = model.predict([images[n]], argmax=False)

# Print the predicted segmentation onto the image.
segmented_image = print_segmentation_onto_image(images[n], prediction, color_map=TRAINIDS_TO_RGBA_DICT)

plt.figure(figsize=(20,14))
plt.imshow(segmented_image)

In [ ]:
model.close()